# Bert

## Imports

In [ ]:
%pip install dotenv

In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments
import os
import wandb

IS_PAPERSPACE = os.getcwd().startswith('/notebooks')
dir_env = os.path.join(os.getcwd(), '.env') if IS_PAPERSPACE else os.path.join(os.getcwd(), '..', '.env')
_ = load_dotenv(dotenv_path=dir_env)

## Data

In [ ]:
df_traindata = pd.read_csv("data/train.csv")

train_texts, validation_texts, train_labels, validation_labels = train_test_split(
    df_traindata['text'].tolist(), 
    df_traindata['sentiment'].tolist(), 
    test_size=0.2, 
    random_state=42
)

df_testdata = pd.read_csv("data/test.csv")
test_texts = df_testdata['text'].tolist()
test_labels = df_testdata['sentiment'].tolist()

## Vectorization

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
validation_encodings = tokenizer(validation_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) 
            for key, val in self.encodings.items()
        } | {
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
validation_dataset = TweetDataset(validation_encodings, validation_labels)
test_dataset = TweetDataset(test_encodings, test_labels)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

try:
    os.environ["WANDB_API_KEY"] = os.getenv('API_KEY_WANDB')
    wandb.login(key=os.getenv('API_KEY_WANDB'))

    os.environ["WANDB_PROJECT"] = "nlp-lantsch-schmassmann-wigger"
    os.environ["WANDB_ENTITY"] = "nlp-lantsch-schmassmann-wigger"
    os.environ["WANDB_WATCH"] = "all"
    
    wandb.init(
        project=os.getenv('WANDB_PROJECT'),
        entity=os.getenv('WANDB_ENTITY'),
        config={
            "learning_rate": 2e-5,
            "epochs": 3,
            "batch_size": 16
        }
    )

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./logs",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        save_total_limit=1,
        report_to=["wandb"] 
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        compute_metrics=compute_metrics
    )


    trainer.train()
    trainer.evaluate()
    test_results = trainer.predict(test_dataset)

    metrics = compute_metrics((test_results.predictions, test_results.label_ids))

    wandb.log({
        "test_precision": metrics["precision"],
        "test_recall": metrics["recall"],
        "test_f1": metrics["f1"]
    })

    wandb.finish()
except Exception as e:
    wandb.finish()
    raise e